1. What is Delta Lake?

Key Concepts to Teach:

Built on top of Parquet
Supports ACID, schema enforcement, versioning
Works best on Databricks and supports MERGE, UPDATE, DELETE


In [0]:
data = [("101", "Alice", "2023-11-01", 1000), ("102", "Bob", "2023-11-01", 2000)]
cols = ["order_id", "customer", "order_date", "amount"]

df = spark.createDataFrame(data, cols)
display(df)

In [0]:
df.write.format('delta').mode('overwrite').saveAsTable('bronze_layer')

In [0]:
%sql
describe history bronze_layer

Layers in Architecture (Medallion):

Bronze ➝ Raw data

Silver ➝ Cleaned + enriched data

Gold ➝ Aggregated insights

In [0]:
# Bronze layer - Ingest from RAW CSV

df_raw = spark.read.option("header", True).csv("/databricks-datasets/retail-org/customers/")

# Rename duplicate columns
for col in df_raw.columns:
    if df_raw.columns.count(col) > 1:
        df_raw = df_raw.withColumnRenamed(col, col + "_duplicate")

df_raw.write.format("delta").mode("overwrite").saveAsTable("customers1")
display(df_raw)

Silver layer

In [0]:
# Silver - Clean and Standard - doesn't seem to work on free edition
df_bronze = spark.read.format("delta").load("/mnt/delta/customers1")

# Display the schema to check the column names
df_bronze.printSchema()

from pyspark.sql.functions import col, lower, trim

# Assuming the correct column name is 'customer_email' instead of 'email'
df_silver = df_bronze.withColumn("customer_name", lower(trim(col("customer_name")))) \
                     .dropna(subset=["customer_id", "customer_name"])

df_silver.write.format("delta").mode("overwrite").saveAsTable("customers2")

display(df_silver)

🟡 Gold – Aggregated Insights

In [0]:
df_silver = spark.read.format("delta").load("/tmp/lakehouse/silver/customers")

df_gold = df_silver.groupBy("state").count()
df_gold.write.format("delta").mode("overwrite").saveAsTable("customer_summary")

In [0]:
%sql
describe history customer_summary